# Drug repurposing with DeepPurpose

- The input to the model is a drug target pair, where drug uses the simplified molecular-input line-entry system (SMILES) string and target uses the amino acid sequence.

- The output is a score indicating the binding activity of the drug target pair.

Tutorial: https://github.com/kexinhuang12345/DeepPurpose/blob/master/Tutorial_1_DTI_Prediction.ipynb

## Objective

1. Find the amino acid sequence of a target known to be involved in a disease (e.g. Alzheimer `MONDO:0004975`)
2. Run the model to get drugs that could potentially bind with Alzheimer target

## Training the model

In [1]:
from DeepPurpose import utils, dataset
from DeepPurpose import DTI as models
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Loading dataset
X_drugs, X_targets, y = dataset.load_process_DAVIS(
    path='./data', 
    binary=False, 
    convert_to_log=True, 
    threshold = 30
)

Beginning Processing...
Beginning to extract zip file...
Default set to logspace (nM -> p) for easier regression
Done!


In [3]:
# Data encoding and split
drug_encoding, target_encoding = 'MPNN', 'CNN'
#drug_encoding, target_encoding = 'Morgan', 'Conjoint_triad'

train, val, test = utils.data_process(
    X_drugs, X_targets, y,
    drug_encoding, target_encoding, 
    split_method='random',
    frac=[0.7,0.1,0.2],
    random_seed = 1
)
train.head(1)

Drug Target Interaction Prediction Mode...
in total: 30056 drug-target pairs
encoding drug...
unique drugs: 68
encoding protein...
unique target sequence: 379
splitting dataset...
Done.


,SMILES,Target Sequence,Label,drug_encoding,target_encoding
0,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.0,"[[[tensor(1.), tensor(0.), tensor(0.), tensor(...","[P, F, W, K, I, L, N, P, L, L, E, R, G, T, Y, ..."


In [4]:
# Model configuration generation
config = utils.generate_config(
    drug_encoding = drug_encoding, 
    target_encoding = target_encoding, 
    cls_hidden_dims = [1024,1024,512], 
    train_epoch = 5, 
    LR = 0.001, 
    batch_size = 128,
    hidden_dim_drug = 128,
    mpnn_hidden_size = 128,
    mpnn_depth = 3, 
    cnn_target_filters = [32,64,96],
    cnn_target_kernels = [4,8,12]
)

In [5]:
model = models.model_initialize(**config)

In [ ]:
model.train(train, val, test)

Let's use CPU/s!
--- Data Preparation ---
--- Go for Training ---
Training at Epoch 1 iteration 0 with loss 31.2962. Total time 0.00055 hours


In [ ]:
from trapi_predict_kit import save

# Save the pre-trained model
save(
    model=model,
    path="models/deeppurpose",
    sample_data=train
)

# Model Prediction and Repuposing/Screening

In [ ]:
X_drug = ['CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC=C4)N']
X_target = ['MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQVTVDEVLAEGGFAIVFLVRTSNGMKCALKRMFVNNEHDLQVCKREIQIMRDLSGHKNIVGYIDSSINNVSSGDVWEVLILMDFCRGGQVVNLMNQRLQTGFTENEVLQIFCDTCEAVARLHQCKTPIIHRDLKVENILLHDRGHYVLCDFGSATNKFQNPQTEGVNAVEDEIKKYTTLSYRAPEMVNLYSGKIITTKADIWALGCLLYKLCYFTLPFGESQVAICDGNFTIPDNSRYSQDMHCLIRYMLEPDPDKRPDIYQVSYFSFKLLKKECPIPNVQNSPIPAKLPEPVKASEAAAKKTQPKARLTDPIPTTETSIAPRQRPKAGQTQPNPGILPIQPALTPRKRATVQPPPQAAGSSNQPGLLASVPQPKPQAPPSQPLPQTQAKQPQAPPTPQQTPSTQAQGLPAQAQATPQHQQQLFLKQQQQQQQPPPAQQQPAGTFYQQQQAQTQQFQAVHPATQKPAIAQFPVVSQGGSQQQLMQNFYQQQQQQQQQQQQQQLATALHQQQLMTQQAALQQKPTMAAGQQPQPQPAAAPQPAPAQEPAIQAPVRQQPKVQTTPPPAVQGQKVGSLTPPSSPKTQRAGHRRILSDVTHSAVFGVPASKSTQLLQAAAAEASLNKSKSATTTPSGSPRTSQQNVYNPSEGSTWNPFDDDNFSKLTAEELLNKDFAKLGEGKHPEKLGGSAESLIPGFQSTQGDAFATTSFSAGTAEKRKGGQTVDSGLPLLSVSDPFIPLQVPDAPEKLIEGLKSPDTSLLLPDLLPMTDPFGSTSDAVIEKADVAVESLIPGLEPPVPQRLPSQTESVTSNRTDSLTGEDSLLDCSLLSNPTTDLLEEFAPTAISAPVHKAAEDSNLISGFDVPEGSDKVAEDEFDPIPVLITKNPQGGHSRNSSGSSESSLPNLARSLLLVDQLIDL']
y = [7.365]
X_pred = utils.data_process(
    X_drug, X_target, y,
    drug_encoding, target_encoding, 
    split_method='no_split'
)
y_pred = model.predict(X_pred)
print('The predicted score is ' + str(y_pred))